# croppedImageSender - docs and install

Interactive cropping tool to define region of interest on a video frame and 
send the video frames to the Streams application.

This is the cropping tool...
- https://openbits.app/posts/python-interactive-cropping/

You need to install it:
```
pip install interactivecrop
```


In [1]:
# The orignal test code
# crop(sample_images, sample_names, optimize=True, continuous_update=True)

## Specify video that will be cropped and analyized. 
**StaticVideo** should point to the video that is to analyized.


In [2]:
StaticVideo = '/Users/siegenth/Data/airportGate.mp4'

## import all the support components

In [3]:
from interactivecrop.interactivecrop import main as crop
from interactivecrop.samples import sample_images, sample_names

In [4]:
""""
Send video, frame-by-frame to Kafka interface
- Frame is encoded into ascii so no one gets upset with the data.
- Frame will be decomposed into chunks of 'CHUNK_SIZE'. When debugging found Kafka would not send message if it went over threshold.
- Receiving test notebook VideoRcvKafka
- The Steams application VideoRcvKafka recieves the encode image and scores it with Model.

"""
import kafka
import os
import sys
import json
import base64
import ssl
import time
import datetime
import io
from PIL import Image
import logging
import cv2
import matplotlib.pyplot as plt
import numpy as np
if '../juypter' not in sys.path:
    sys.path.insert(0, '../juypter')
import credential
import streams_aid as aid

logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))
# img_encoded = str(base64.b64encode(response.content).decode("utf-8"))
# img_encoded = str(base64.b64encode(img).decode('utf-8'))


def bts_to_img(bts):
    buff = np.fromstring(bts, np.uint8)
    buff = buff.reshape(1, -1)
    img = cv2.imdecode(buff, cv2.IMREAD_COLOR)
    return img


def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


def encode_img(img):
    """must be easier way"""
    with io.BytesIO() as output:
        img.save(output, format="JPEG")
        contents = output.getvalue()
    return base64.b64encode(contents).decode('ascii')


def decode_img(bin64):
    """must be easier way"""
    img = Image.open(io.BytesIO(base64.b64decode(bin64)))
    return img



## Get and image from video and set region of interest. 


### collect one frame from the video

In [5]:
def collect_frames(video_url, frame_count=1, frame_modulo=24, debug=False):
    """collect a set of frames from the video to work out the cropping region. 
    Notes:
        - pull out the frames based upon the modulo and frame_count
        - the correct way, find frames that hav signficant difference between each
        - now 
    
    """
    frames = []
    """get the crop region for a video.
    []  pull up some frames...
    [x] - send frames to cropper
    [x] - get cropper regionquick

    :param kafka_prod: the handle to sent out messages on kafka
    :param frame_modulo: send every x frames
    :param send_wait: after sending a frame wait time
    :param debug: decode image and write out to verify
    :return: None

    """
    frame_num = 0

    cap = cv2.VideoCapture(video_url)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret is False:
            break
        frame_num += 1
        if not(frame_num % frame_modulo):            
            if debug:
                image_encoded =encode_img(Image.fromarray(frame, 'RGB'))
                # debugging - render what we will send.
                img_raw = decode_img(image_encoded)
                plt.imshow(img_raw)
                plt.show()
            # break down frame into chunks

            frames.append(frame)

        if frame_count <= len(frames):
            break
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return frames

secs = 30 
frames = collect_frames(video_url=StaticVideo,frame_modulo=30*secs,
              frame_count=1, debug=False)
print("Collected {} frames at the {} second mark.".format(len(frames), secs))

Collected 1 frames at the 30 second mark.


### Use the collected frame to define a crop region. 

In [6]:
SHAPE = None
def grabCropShape(image_name, shape):
    global SHAPE
    SHAPE = shape
    print("set SHAPE ", image_name, shape, flush=True)

### Press the 'Save Crop Size' button to capture the crop region.

In [7]:
crop(frames, callback=grabCropShape)

GridBox(children=(Dropdown(description='Img Name:', layout=Layout(grid_area='im_selector', width='100%'), opti…

interactive(children=(Button(description='Save Crop Sizes', style=ButtonStyle()), Output()), _dom_classes=('wi…

### Verify that the captured region is what you expected
- verify what we collected
- **RegionOfInterest** is the cropping specification that will be applied to frame sent

In [8]:
image_encoded =encode_img(Image.fromarray(frames[0], 'RGB'))
img_raw = decode_img(image_encoded)
print("Image size : {} crop region : {} ".format(img_raw.size, SHAPE.size))
RegionOfInterest = (SHAPE.size[0],
                        SHAPE.size[1],
                        SHAPE.size[0]+SHAPE.size[2],
                        SHAPE.size[1]+SHAPE.size[3])
print("regionOfInterest:",RegionOfInterest)
cropped = img_raw.crop(RegionOfInterest)

plt.imshow(cropped)
plt.show()

AttributeError: 'NoneType' object has no attribute 'size'

## Send Cropped Region....

In [ ]:
def kafka_producer(credentials):
    """
    Open the connection to the kafka producer
    :param credentials:
    :return: kafka producer

    Request is responsilbe for closing producer.
    """
    prod = None
    while prod is None:
        try:
            prod = kafka.KafkaProducer(bootstrap_servers=credentials["kafka_brokers_sasl"],
                                       security_protocol="SASL_SSL",
                                       sasl_mechanism="PLAIN",
                                       sasl_plain_username=credentials["user"],
                                       sasl_plain_password=credentials["api_key"],
                                       ssl_cafile=ssl.get_default_verify_paths().cafile)

        except kafka.errors.NoBrokersAvailable:
            logging.warning("No Brokers Available. Retrying ...")
            time.sleep(1)
            prod = None
    return prod

In [ ]:
CHUNK_SIZE = 100000         # maximum number of bytes to transmit at a time

def video_kafka(video_url, regionOfInterest, kafka_prod, kafka_topic='VideoFrame', frame_modulo=24, send_wait=.25, debug=False):
    """Send video via Kafka

    :param video_url: url of video to pull in and send
    :param kafka_prod: the handle to sent out messages on kafka
    :param frame_modulo: send every x frames
    :param send_wait: after sending a frame wait time
    :param debug: decode image and write out to verify
    :return: None

    """
    frame_num = 0

    cap = cv2.VideoCapture(video_url)
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret is False:
            break
        frame_num += 1
        if not(frame_num % frame_modulo):
            # crop each frame before sending it.
            orginal_encoded =encode_img(Image.fromarray(frame, 'RGB'))
            img_raw = decode_img(orginal_encoded)
            cropped = img_raw.crop(regionOfInterest)
            image_encoded = encode_img(cropped)
            
            
            if debug:
                # debugging - render what we will send.
                img_raw = decode_img(image_encoded)
                plt.imshow(img_raw)
                plt.show()
            # break down frame into chunks
            chunks = [image_encoded[i * CHUNK_SIZE:(i + 1) * CHUNK_SIZE] for i in
                      range((len(image_encoded) + CHUNK_SIZE - 1) // CHUNK_SIZE)]
            # send the chunks.
            for idx, chunk in enumerate(chunks):
                logging.debug("chunking - {}  #chunks :{} idx:{} len(chunk):{}".format(video_url, len(chunks), idx, len(chunk)))
                chunk_content = {'video': video_url,
                       'frame': frame_num,
                       'chunk_idx':idx,
                       'chunk_total':len(chunks),
                       'timestamp': datetime.datetime.utcnow().isoformat() + 'Z',
                       'data': chunk
                }
                kafka_prod.send(kafka_topic, value=json.dumps(chunk_content).encode('utf-8'))
            ## finish the frame frame
            chunk_complete = {'video': video_url,
                   'frame': frame_num,
                    'chunk_idx': len(chunks),
                    'chunk_total': len(chunks),
                   'timestamp': datetime.datetime.utcnow().isoformat() + 'Z',
                   'data': ""
                   }
            logging.info("Transmit frame #{}".format(chunk_content["frame"]))
            kafka_prod.send(kafka_topic, value=json.dumps(chunk_complete).encode('utf-8'))
            time.sleep(send_wait)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    return

In [ ]:
@aid.catchInterrupt
def videoStream(topic="VideoFrame", videoUrl=None, regionOfInterest=None):
    creds = json.loads(credential.magsEventStream)
    prod = kafka_producer(creds,)
    video_kafka(videoUrl, regionOfInterest, prod,  kafka_topic=topic, send_wait=1, frame_modulo=24, debug=False)
    prod.close()

TOPIC="VideoFrame"    

videoStream(topic=TOPIC, videoUrl=StaticVideo, regionOfInterest=RegionOfInterest)